# Snowflake Iceberg V3 VARIANT Demo

This notebook demonstrates:
- Creating Iceberg V3 tables with VARIANT columns
- Loading JSON data into VARIANT columns
- Querying VARIANT data using semi-structured notation
- Extracting data using AI_EXTRACT()
- Redacting PII using AI_REDACT()

## Prerequisites
- Snowflake setup SQL has been run successfully
- JSON files have been uploaded to the JSON_STAGE
- You're running this notebook in Snowflake with access to the V3_DEMO_ROLE


## Setup: Set Session Context

First, set the warehouse, role, database, and schema we'll be working with.

**Update these values to match your setup:**


In [ ]:
-- Set your warehouse name
USE WAREHOUSE YOUR_WAREHOUSE;

-- Set the role created by the setup script
USE ROLE FIRST_LAST_DATA_ENGINEER;

-- Set database (update with your first_last name)
USE DATABASE FIRST_LAST_ICEBERG_V3_DEMO;

-- Start in the RAW schema
USE SCHEMA RAW;

SELECT CURRENT_WAREHOUSE(), CURRENT_ROLE(), CURRENT_DATABASE(), CURRENT_SCHEMA() AS context_info;

## Step 1: Verify JSON Files in Stage

List the JSON files you uploaded to the internal stage:


In [ ]:
LIST @JSON_STAGE;

## Step 2: Try Creating Iceberg Table Without V3 (Will Fail)

Let's first try to create an Iceberg table with a VARIANT column without specifying ICEBERG_VERSION = 3.

**This should fail** because VARIANT is only supported in Iceberg format version 3:


In [ ]:
CREATE OR REPLACE ICEBERG TABLE CUSTOMER_EVENTS (
    event_data VARIANT
)
    CATALOG = 'SNOWFLAKE'
    EXTERNAL_VOLUME = 'YOUR_EXTERNAL_VOLUME'
    BASE_LOCATION = 'YOUR_DATABASE/YOUR_SCHEMA/CUSTOMER_EVENTS';

-- Expected error: VARIANT is not supported with Iceberg version 2

## Step 3: Create Iceberg V3 Table with VARIANT (Success!)

Now let's create the table correctly by specifying `ICEBERG_VERSION = 3`:


In [ ]:
CREATE OR REPLACE ICEBERG TABLE CUSTOMER_EVENTS (
    event_data VARIANT
)
    CATALOG = 'SNOWFLAKE'
    EXTERNAL_VOLUME = 'YOUR_EXTERNAL_VOLUME'
    BASE_LOCATION = 'YOUR_DATABASE/RAW/CUSTOMER_EVENTS'
    ICEBERG_VERSION = 3
;

## Step 4: Load JSON data into Iceberg Variant

Load data from JSON files into an Iceberg table with a Variant column.


In [ ]:
COPY INTO CUSTOMER_EVENTS
    FROM @JSON_STAGE
    FILE_FORMAT = (TYPE = 'JSON')
    PATTERN = '.*customer_events_.*\\.json'
    ON_ERROR = 'CONTINUE';

## Step 5: Query Using Semi-Structured Notation

Extract specific fields from the VARIANT column using Snowflake's dot notation:


In [ ]:
SELECT * FROM CUSTOMER_EVENTS LIMIT 10;

In [ ]:
SELECT 
    event_data:event_id::STRING AS event_id,
    event_data:timestamp::TIMESTAMP AS event_timestamp,
    event_data:event_type::STRING AS event_type,
    event_data:customer.name::STRING AS customer_name,
    event_data:customer.email::STRING AS customer_email,
    event_data:customer.phone::STRING AS customer_phone
FROM CUSTOMER_EVENTS
ORDER BY event_timestamp
LIMIT 10;

## Step 6: Use AI_REDACT() to Protect PII

Use Snowflake Cortex AI's `AI_REDACT()` function to automatically identify and redact PII.

Reference: https://docs.snowflake.com/en/user-guide/snowflake-cortex/redact-pii


In [ ]:
CREATE OR REPLACE ICEBERG TABLE REDACTED.CUSTOMER_EVENTS_REDACTED
    CATALOG = 'SNOWFLAKE'
    EXTERNAL_VOLUME = 'EXVOL'
    BASE_LOCATION = 'NOV18/REDACTED/CUSTOMER_EVENTS_REDACTED/'
    ICEBERG_VERSION = 3
AS
SELECT 
    PARSE_JSON(AI_REDACT(event_data)) AS event_data,
    event_data:event_id::STRING AS event_id,
    event_data:event_type::STRING AS event_type,
    event_data:timestamp::TIMESTAMP(6) AS event_timestamp
FROM RAW.CUSTOMER_EVENTS;

In [ ]:
SELECT *
FROM REDACTED.CUSTOMER_EVENTS_REDACTED
LIMIT 5;